In [192]:
#Importing Libraries
import glob
from keras.models import Sequential, load_model
import numpy as np
import pandas as pd
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import keras as k
import seaborn as sns                     # data visualization library based on matplotlib
import sklearn                            # for random forests, and k-neighbours etc 

sns.set(rc={'figure.figsize':(12,8)})

In [193]:
#Load the data
from google.colab import files
uploaded = files.upload()

df = pd.read_csv('Anaemia1.csv')

#Print the first 10 rows
df.head(10)

Saving Anaemia1.csv to Anaemia1 (10).csv


,S. No.,Age,Sex,Haemoglobin,MCH,MCHC,MCV,RBC,Platlets,IDENTIFICATION
0,1,28,0,14.9,22.7,29.1,83.7,5.66,128.3,Not Anaemia
1,2,41,0,15.9,25.4,28.3,72.0,4.78,419.0,Not Anaemia
2,3,40,1,9.0,21.5,29.6,71.2,4.65,325.0,Anaemia
3,4,76,0,14.9,16.0,31.4,87.5,4.24,264.0,Not Anaemia
4,5,20,1,14.7,22.0,28.2,99.5,4.14,196.0,Not Anaemia
5,6,24,0,11.6,22.3,30.9,74.5,4.29,233.0,Anaemia
6,7,28,1,12.7,19.5,28.9,82.9,4.98,213.0,Anaemia
7,8,14,0,12.7,28.5,28.2,92.3,4.97,229.0,Anaemia
8,9,16,0,14.1,29.7,30.5,75.2,4.16,211.0,Not Anaemia
9,10,62,0,14.9,25.8,31.3,82.9,5.25,151.0,Not Anaemia


In [194]:
df.head()

,S. No.,Age,Sex,Haemoglobin,MCH,MCHC,MCV,RBC,Platlets,IDENTIFICATION
0,1,28,0,14.9,22.7,29.1,83.7,5.66,128.3,Not Anaemia
1,2,41,0,15.9,25.4,28.3,72.0,4.78,419.0,Not Anaemia
2,3,40,1,9.0,21.5,29.6,71.2,4.65,325.0,Anaemia
3,4,76,0,14.9,16.0,31.4,87.5,4.24,264.0,Not Anaemia
4,5,20,1,14.7,22.0,28.2,99.5,4.14,196.0,Not Anaemia


In [195]:
df.shape

(199, 10)

In [196]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   S. No.          199 non-null    int64  
 1   Age             199 non-null    int64  
 2   Sex             199 non-null    int64  
 3   Haemoglobin     199 non-null    float64
 4   MCH             199 non-null    float64
 5   MCHC            199 non-null    float64
 6   MCV             199 non-null    float64
 7   RBC             199 non-null    float64
 8   Platlets        199 non-null    float64
 9   IDENTIFICATION  199 non-null    object 
dtypes: float64(6), int64(3), object(1)
memory usage: 15.7+ KB


In [197]:
df.isnull().sum()

S. No.            0
Age               0
Sex               0
Haemoglobin       0
MCH               0
MCHC              0
MCV               0
RBC               0
Platlets          0
IDENTIFICATION    0
dtype: int64

In [198]:
df.describe()

,S. No.,Age,Sex,Haemoglobin,MCH,MCHC,MCV,RBC,Platlets
count,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000
mean,100.000000,42.341709,0.402010,13.554774,22.611055,30.104523,84.294472,4.263970,214.448744
std,57.590508,18.788666,0.491541,2.002540,3.998810,2.121375,12.006880,0.817041,100.020943
min,1.000000,11.000000,0.000000,6.900000,11.400000,11.400000,11.400000,1.960000,10.000000
25%,50.500000,27.500000,0.000000,11.850000,19.300000,29.150000,76.700000,3.715000,150.000000
50%,100.000000,40.000000,0.000000,13.400000,22.300000,30.300000,84.400000,4.330000,198.000000
75%,149.500000,57.000000,1.000000,15.200000,25.450000,31.300000,92.900000,4.770000,258.500000
max,199.000000,89.000000,1.000000,16.900000,30.000000,32.500000,101.500000,6.900000,589.000000


In [199]:
df['IDENTIFICATION'].value_counts()

Not Anaemia    123
Anaemia         76
Name: IDENTIFICATION, dtype: int64

In [200]:
df.groupby('IDENTIFICATION').mean()

,S. No.,Age,Sex,Haemoglobin,MCH,MCHC,MCV,RBC,Platlets
IDENTIFICATION,,,,,,,,,
Anaemia,100.394737,42.934211,0.355263,11.580263,22.643421,30.040789,82.611842,4.202105,209.302632
Not Anaemia,99.756098,41.975610,0.430894,14.774797,22.591057,30.143902,85.334146,4.302195,217.628455


In [201]:
#Transform the non-numeric data in the columns
for column in df.columns:
  if df[column].dtype == np.number:
    continue
  df[column] = LabelEncoder().fit_transform( df[column])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  This is separate from the ipykernel package so we can avoid doing imports until


In [202]:
#Feature Scaling 
#Min-Max scaler method scales the dataset so that all the input features lie between 0 and 1
X_scaler = MinMaxScaler()
X_scaler.fit(X)
column_names = X.columns
X[column_names] = X_scaler.transform(X)

In [203]:
X = df.drop(columns=['S. No.','IDENTIFICATION'], axis=1)
Y = df['IDENTIFICATION']

In [204]:
print(X)

     Age        Sex    Haemoglobin   MCH  MCHC   MCV   RBC  Platlets
0           16      0         14.9  22.7  29.1  83.7  5.66     128.3
1           28      0         15.9  25.4  28.3  72.0  4.78     419.0
2           27      1          9.0  21.5  29.6  71.2  4.65     325.0
3           58      0         14.9  16.0  31.4  87.5  4.24     264.0
4            8      1         14.7  22.0  28.2  99.5  4.14     196.0
..         ...    ...          ...   ...   ...   ...   ...       ...
194         19      0         14.8  18.8  31.9  77.5  4.46     152.0
195         56      0         12.7  23.3  30.3  92.8  4.91     266.0
196         20      0         14.6  26.3  32.0  90.7  4.40     105.0
197          9      1         16.4  27.9  29.2  87.9  4.03     289.0
198          5      0         16.8  24.3  31.5  92.0  5.04     150.0

[199 rows x 8 columns]


In [205]:
print(Y)

0      1
1      1
2      0
3      1
4      1
      ..
194    1
195    1
196    1
197    1
198    1
Name: IDENTIFICATION, Length: 199, dtype: int64


In [206]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [207]:
print(X.shape, X_train.shape, X_test.shape)

(199, 8) (159, 8) (40, 8)


In [208]:
scaler = StandardScaler()

In [209]:
scaler.fit(X_train)

StandardScaler()

In [210]:
X_train = scaler.transform(X_train)

X_test = scaler.transform(X_test)

In [211]:
print(X_train)

[[-0.03676132  1.28452326  0.0754402  ... -1.26353128 -0.10658896
   0.08593312]
 [ 0.31046933  1.28452326  0.68278157 ...  0.63735403  0.25839979
   1.85404789]
 [ 0.88918707 -0.77849894  0.98645225 ...  1.3800255   2.11984244
  -0.29889185]
 ...
 [-1.36781212  1.28452326 -1.44291322 ... -0.5827491  -0.56890805
   0.54356283]
 [-1.13632502 -0.77849894  0.22727554 ... -0.29098531  1.04920876
   0.56436418]
 [ 0.25259755 -0.77849894  0.17666376 ... -0.54738379 -0.15525413
  -0.53810738]]


In [212]:
#Build the model
model = Sequential()
model.add( Dense(256, input_dim = len(X.columns), kernel_initializer= k.initializers.random_normal(seed=13), activation='relu'))
model.add( Dense(1, activation = 'hard_sigmoid'))

In [213]:
#Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [214]:
#Train the model
history = model.fit(X_train, Y_train, epochs = 500, batch_size = X_train.shape[0])

Epoch 1/500
1/1 [==============================] - 0s 382ms/step - loss: 0.6911 - accuracy: 0.5283
Epoch 2/500
1/1 [==============================] - 0s 5ms/step - loss: 0.6831 - accuracy: 0.5912
Epoch 3/500
1/1 [==============================] - 0s 5ms/step - loss: 0.6754 - accuracy: 0.6038
Epoch 4/500
1/1 [==============================] - 0s 5ms/step - loss: 0.6678 - accuracy: 0.6415
Epoch 5/500
1/1 [==============================] - 0s 7ms/step - loss: 0.6604 - accuracy: 0.6730
Epoch 6/500
1/1 [==============================] - 0s 5ms/step - loss: 0.6531 - accuracy: 0.6792
Epoch 7/500
1/1 [==============================] - 0s 6ms/step - loss: 0.6459 - accuracy: 0.6981
Epoch 8/500
1/1 [==============================] - 0s 12ms/step - loss: 0.6389 - accuracy: 0.7233
Epoch 9/500
1/1 [==============================] - 0s 6ms/step - loss: 0.6320 - accuracy: 0.7233
Epoch 10/500
1/1 [==============================] - 0s 4ms/step - loss: 0.6252 - accuracy: 0.7296
Epoch 11/500
1/1 [========

Building a Predictive System

In [228]:
input_data = (40,1,9,21.5,29.6,71.2,4.65,325)

# changing input data to a numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the numpy array
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

# standardize the data
std_data = scaler.transform(input_data_reshaped)

prediction = model.predict(std_data)
print(prediction)


if (prediction[0]==0):
  print("The Person has Anaemia")

else:
  print("The Person is healthy")


[[0.]]
The Person has Anaemia


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
